In [1]:
import pandas as pd
import numpy as np
import pickle
import torch

In [2]:
train = pd.read_csv('../Data/train.csv')
#test = pd.read_csv('../Data/test.csv')
structures = pd.read_csv('../Data/structures.csv')

In [3]:
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

In [4]:
train = map_atom_info(train, 0)
train = map_atom_info(train, 1)

In [5]:
centers = (train.loc[:,['x_0','y_0','z_0']].values + train.loc[:,['x_1','y_1','z_1']].values)/2

In [6]:
vector = train.loc[:,['x_0','y_0','z_0']].values - train.loc[:,['x_1','y_1','z_1']].values

In [7]:
norm = np.linalg.norm(vector,axis=1)[:,None]
vector = vector/norm

In [3]:
train#.loc[train.molecule_name=='dsgdb9nsd_000001']

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.807600
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.257000
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.254800
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.254300
4,4,dsgdb9nsd_000001,2,0,1JHC,84.807400
5,5,dsgdb9nsd_000001,2,3,2JHH,-11.254100
6,6,dsgdb9nsd_000001,2,4,2JHH,-11.254800
7,7,dsgdb9nsd_000001,3,0,1JHC,84.809300
8,8,dsgdb9nsd_000001,3,4,2JHH,-11.254300
9,9,dsgdb9nsd_000001,4,0,1JHC,84.809500


In [13]:
structures#.loc[structures.molecule_name=='dsgdb9nsd_000001']

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397
5,dsgdb9nsd_000002,0,N,-0.040426,1.024108,0.062564
6,dsgdb9nsd_000002,1,H,0.017257,0.012545,-0.027377
7,dsgdb9nsd_000002,2,H,0.915789,1.358745,-0.028758
8,dsgdb9nsd_000002,3,H,-0.520278,1.343532,-0.775543
9,dsgdb9nsd_000003,0,O,-0.034360,0.977540,0.007602


In [12]:
structures[['C', 'F', 'H', 'N', 'O']] = pd.get_dummies(structures.atom)

In [13]:
structures_gb = structures.groupby(['molecule_name'])

In [14]:
structures_gb = {k:v.loc[:,['x','y','z','C', 'F', 'H', 'N', 'O']].values for k,v in structures_gb}

In [62]:
def create(mol,c,v):
    X = mol[:,:3] - c
    r = np.matmul(X,v)
    R = np.linalg.norm(X - r[:,None]*v,axis=1)
    out = np.concatenate([r[:,None],R[:,None],mol[:,3:],np.broadcast_to(d,mol.shape[0])[:,None]],1)
    return out

In [77]:
feature_list = []
for i in range(train.shape[0]):
    mol = structures_gb[train.iloc[i]['molecule_name']]
    c,v,d = centers[i],vector[i],norm[i]
    feature_list.append(create(mol,c,v))

In [85]:
def numpy2torch(x):
    return torch.from_numpy(x.astype(np.float32))

In [89]:
feature_list = [numpy2torch(i) for i in feature_list]

In [90]:
with open('../Data/train_data_attention_node.pickle', 'wb') as handle:
    pickle.dump(feature_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [95]:
edge_X = torch.from_numpy(np.concatenate([pd.get_dummies(train.type).values,norm],1).astype(np.float32))

In [99]:
with open('../Data/train_data_attention_edge.pickle', 'wb') as handle:
    pickle.dump(edge_X, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
ind = torch.from_numpy(train.loc[:,['atom_index_0','atom_index_1']].values)

with open('../Data/train_data_ind.pickle', 'wb') as handle:
    pickle.dump(ind, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [101]:
edge_y = torch.from_numpy(train.scalar_coupling_constant.values.astype(np.float32))

In [103]:
with open('../Data/train_data_attention_edge_y.pickle', 'wb') as handle:
    pickle.dump(edge_y, handle, protocol=pickle.HIGHEST_PROTOCOL)